In [1]:
import pandas as pd
import numpy as np
import pickle
import gzip
import os
import subprocess
from os import path
import fnmatch
import subprocess
import scipy.io.wavfile as wav
from sklearn.metrics import accuracy_score
from import_timit import read_transcript
from import_timit import compute_mfcc

In [2]:
accuracy_list = []
target = "./TIMIT/TEST"
full_wavs = []

for root, dirnames, filenames in os.walk(target):
    for filename in fnmatch.filter(filenames, "*.WAV"):
        sph_file = os.path.join(root, filename)
        wav_file = os.path.join(root, filename)[:-4] + "_rif.wav"
        full_wavs.append(wav_file)
        #print("converting {} to {}".format(sph_file, wav_file))
        subprocess.check_call(["sox", sph_file, wav_file])

print("Preprocessing Complete")

Preprocessing Complete


# Frame Level Accuracy

In [3]:
#Test with GMM with 64 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/064"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [4]:
print("Testing with GMM model with 64 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 64 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 64 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  1.2345679012345678 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  1.9607843137254901 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  1.4981273408239701 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  0.6756756756756757 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  2.8112449799196786 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  2.898550724637681 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  2.6936026936026933 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  2.5559105431309903 %

Test File:  ./TIMIT/TEST/DR4/MTLS0/SI2000_rif.wav
Fram

Frame Level Accuracy:  2.4242424242424243 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SI1088_rif.wav
Frame Level Accuracy:  1.520912547528517 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  2.284263959390863 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  1.2096774193548387 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  2.5 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  2.0408163265306123 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  1.4084507042253522 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  3.4700315457413247 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  0.9345794392523363 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  1.694915254237288 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  4.2792792792792795 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/

Frame Level Accuracy:  5.714285714285714 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SI1502_rif.wav
Frame Level Accuracy:  0.8928571428571428 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  1.3953488372093024 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  4.046242774566474 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  1.3245033112582782 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  2.697841726618705 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  0.9966777408637874 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  1.6181229773462782 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  5.585106382978723 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  2.3880597014925375 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX

Frame Level Accuracy:  1.893939393939394 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SI1850_rif.wav
Frame Level Accuracy:  3.896103896103896 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  1.8656716417910446 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  2.941176470588235 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  0.41841004184100417 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  3.041825095057034 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  2.422145328719723 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  1.2096774193548387 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  3.77906976744186 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame Level Accuracy:  0.5050505050505051 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI20

Frame Level Accuracy:  2.4875621890547266 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX109_rif.wav
Frame Level Accuracy:  1.3377926421404682 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  4.3478260869565215 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  2.314814814814815 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  3.070175438596491 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  0.9259259259259258 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  1.694915254237288 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  1.1267605633802817 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX383_rif.wav
Frame Level Accuracy:  1.486988847583643 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/

...

Frame Level Accuracy:  3.0534351145038165 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX283_rif.wav
Frame Level Accuracy:  1.0204081632653061 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX138_rif.wav
Frame Level Accuracy:  3.7453183520599254 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI858_rif.wav
Frame Level Accuracy:  1.2953367875647668 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1026_rif.wav
Frame Level Accuracy:  2.366863905325444 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA2_rif.wav
Frame Level Accuracy:  2.73972602739726 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX48_rif.wav
Frame Level Accuracy:  2.4 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA1_rif.wav
Frame Level Accuracy:  7.4324324324324325 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX228_rif.wav
Frame Level Accuracy:  2.2857142857142856 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX408_rif.wav
Frame Level Accuracy:  1.6129032258064515 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX318_rif.wav
Frame Level Accuracy:  4.016064257028113 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1

Frame Level Accuracy:  4.088050314465408 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX126_rif.wav
Frame Level Accuracy:  0.4405286343612335 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX36_rif.wav
Frame Level Accuracy:  3.1948881789137378 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI2210_rif.wav
Frame Level Accuracy:  2.247191011235955 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI1580_rif.wav
Frame Level Accuracy:  0.38022813688212925 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX410_rif.wav
Frame Level Accuracy:  2.05761316872428 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA2_rif.wav
Frame Level Accuracy:  6.83453237410072 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA1_rif.wav
Frame Level Accuracy:  2.416918429003021 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX50_rif.wav
Frame Level Accuracy:  0.3401360544217687 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX230_rif.wav
Frame Level Accuracy:  1.238390092879257 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI950_rif.wav
Frame Level Accuracy:  2.066115702479339 %

Test File:  ./TIMIT/TEST/DR

Frame Level Accuracy:  1.477832512315271 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI934_rif.wav
Frame Level Accuracy:  1.3029315960912053 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI2194_rif.wav
Frame Level Accuracy:  1.201923076923077 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX34_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI653_rif.wav
Frame Level Accuracy:  1.8726591760299627 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX113_rif.wav
Frame Level Accuracy:  2.4390243902439024 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1283_rif.wav
Frame Level Accuracy:  3.1390134529147984 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1423_rif.wav
Frame Level Accuracy:  1.8518518518518516 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SA2_rif.wav
Frame Level Accuracy:  7.623318385650224 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX203_rif.wav
Frame Level Accuracy:  3.1141868512110724 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SA1_rif.wav
Frame Level Accuracy:  5.614035087719298 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/S

Frame Level Accuracy:  2.666666666666667 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA1_rif.wav
Frame Level Accuracy:  0.5899705014749262 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX396_rif.wav
Frame Level Accuracy:  2.710843373493976 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX126_rif.wav
Frame Level Accuracy:  3.802281368821293 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX36_rif.wav
Frame Level Accuracy:  2.4475524475524475 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX304_rif.wav
Frame Level Accuracy:  3.116883116883117 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI2104_rif.wav
Frame Level Accuracy:  2.810304449648712 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA2_rif.wav
Frame Level Accuracy:  4.6875 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA1_rif.wav
Frame Level Accuracy:  3.0737704918032787 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX214_rif.wav
Frame Level Accuracy:  3.7783375314861463 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX394_rif.wav
Frame Level Accuracy:  0.5617977528089888 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX1

Frame Level Accuracy:  2.5316455696202533 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI2079_rif.wav
Frame Level Accuracy:  1.2345679012345678 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX9_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX369_rif.wav
Frame Level Accuracy:  1.9230769230769231 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI1449_rif.wav
Frame Level Accuracy:  1.0723860589812333 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX279_rif.wav
Frame Level Accuracy:  1.282051282051282 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX317_rif.wav
Frame Level Accuracy:  3.7209302325581395 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI587_rif.wav
Frame Level Accuracy:  2.768166089965398 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SA2_rif.wav
Frame Level Accuracy:  2.7522935779816518 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX407_rif.wav
Frame Level Accuracy:  1.5151515151515151 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SA1_rif.wav
Frame Level Accuracy:  4.285714285714286 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX

In [5]:
#Test with GMM with 2 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/002"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [6]:
print("Testing with GMM model with 2 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 2 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 2 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  1.7241379310344827 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  2.4691358024691357 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  2.6143790849673203 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  2.247191011235955 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  1.0135135135135136 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  2.0080321285140563 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  3.864734299516908 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  2.356902356902357 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  2.2364217252396164 %

Test File:  ./TIMIT/TEST/DR4/MTLS0/SI2000_

Frame Level Accuracy:  1.1406844106463878 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  1.2690355329949239 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  0.8064516129032258 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  2.0 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  0.40816326530612246 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  0.7042253521126761 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  4.100946372239748 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  1.2461059190031152 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  1.2711864406779663 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  5.8558558558558556 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283_rif.wav
Frame Level Accuracy:  0.5970149253731344 %

Test File:  ./TIMIT/TEST/DR4/FADG

Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  1.1560693641618496 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  4.304635761589404 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  2.338129496402878 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  5.3156146179401995 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  0.6472491909385114 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  6.382978723404255 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  1.4925373134328357 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX94_rif.wav
Frame Level Accuracy:  2.6755852842809364 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1714_rif.wav
Frame Level Accuracy:  0.3389830508474576 %

Test File:  ./TIMIT/TEST/DR4/FGJD0/S

Frame Level Accuracy:  2.5210084033613445 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  0.8368200836820083 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  1.520912547528517 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  1.7301038062283738 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  2.9585798816568047 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  4.3604651162790695 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame Level Accuracy:  0.5050505050505051 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI2027_rif.wav
Frame Level Accuracy:  0.6514657980456027 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX137_rif.wav
Frame Level Accuracy:  2.331002331002331 %

Test File:  ./TIMIT/TEST/DR4/FSEM0/SX298_rif.wav
Frame Level Accuracy:  0.3676470588235294 %

Test File:  ./TIMIT/TEST/DR4/FSEM0/

Frame Level Accuracy:  1.932367149758454 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  6.976744186046512 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  3.2407407407407405 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  3.389830508474576 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  0.5633802816901409 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX383_rif.wav
Frame Level Accuracy:  0.37174721189591076 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX293_rif.wav
Frame Level Accuracy:  0.38022813688212925 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX23_rif.wav
Frame Level Accuracy:  1.8433179723502304 %

Test File:  ./TIMIT/TEST/DR3/MJBR0/SX11_rif.wav
Frame Level Accuracy:  2.2346368715083798 %

Test File:  ./TIMIT/TEST/DR3/MJBR0/S

...

Frame Level Accuracy:  3.237410071942446 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI2189_rif.wav
Frame Level Accuracy:  5.0761421319796955 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX29_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX389_rif.wav
Frame Level Accuracy:  5.213270142180095 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA2_rif.wav
Frame Level Accuracy:  1.2987012987012987 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA1_rif.wav
Frame Level Accuracy:  11.69811320754717 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI1559_rif.wav
Frame Level Accuracy:  2.083333333333333 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX209_rif.wav
Frame Level Accuracy:  6.25 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX119_rif.wav
Frame Level Accuracy:  5.747126436781609 %

Test File:  ./TIMIT/TEST/DR7/MGRT0/SX10_rif.wav
Frame Level Accuracy:  1.4925373134328357 %

Test File:  ./TIMIT/TEST/DR7/MGRT0/SX100_rif.wav
Frame Level Accuracy:  1.3793103448275863 %

Test File:  ./TIMIT/TEST/DR7/MGRT0/SI1

Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX403_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA2_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA1_rif.wav
Frame Level Accuracy:  2.564102564102564 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX223_rif.wav
Frame Level Accuracy:  4.371584699453552 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX43_rif.wav
Frame Level Accuracy:  4.216867469879518 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI2293_rif.wav
Frame Level Accuracy:  2.5974025974025974 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI1663_rif.wav
Frame Level Accuracy:  2.3696682464454977 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX133_rif.wav
Frame Level Accuracy:  2.272727272727273 %

Test File:  ./TIMIT/TEST/DR6/MDSC0/SX138_rif.wav
Frame Level Accuracy:  3.0 %

Test File:  ./TIMIT/TEST/DR6/MDSC0/SA2_rif.wav
Frame Level Accuracy:  1.4705882352941175 %

Test File:  ./TIMIT/TEST/DR6/MDSC0/SI2298_rif.wav
Frame Level Accuracy:  8.411214953

Frame Level Accuracy:  0.3861003861003861 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX372_rif.wav
Frame Level Accuracy:  0.3676470588235294 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1182_rif.wav
Frame Level Accuracy:  1.6736401673640167 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA2_rif.wav
Frame Level Accuracy:  0.36363636363636365 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI2313_rif.wav
Frame Level Accuracy:  3.6511156186612577 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA1_rif.wav
Frame Level Accuracy:  4.220779220779221 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX192_rif.wav
Frame Level Accuracy:  0.40650406504065045 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX282_rif.wav
Frame Level Accuracy:  1.3937282229965158 %

Test File:  ./TIMIT/TEST/DR6/MPAM1/SX306_rif.wav
Frame Level Accuracy:  2.1834061135371177 %

Test File:  ./TIMIT/TEST/DR6/MPAM1/SX216_rif.wav
Frame Level Accuracy:  1.82648401826484 %

Test File:  ./TIMIT/TEST/DR6/MPAM1/SA2_rif.wav
Frame Level Accuracy:  1.2295081967213115 %

Test File:  ./TIMIT/

Frame Level Accuracy:  2.4154589371980677 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA1_rif.wav
Frame Level Accuracy:  4.761904761904762 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX200_rif.wav
Frame Level Accuracy:  1.8518518518518516 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI2270_rif.wav
Frame Level Accuracy:  4.225352112676056 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1010_rif.wav
Frame Level Accuracy:  2.2277227722772275 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX380_rif.wav
Frame Level Accuracy:  2.91005291005291 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX290_rif.wav
Frame Level Accuracy:  0.9584664536741214 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX20_rif.wav
Frame Level Accuracy:  4.012345679012346 %

Test File:  ./TIMIT/TEST/DR1/FJEM0/SX274_rif.wav
Frame Level Accuracy:  4.1551246537396125 %

Test File:  ./TIMIT/TEST/DR1/FJEM0/SX364_rif.wav
Frame Level Accuracy:  6.420233463035019 %

Test File:  ./TIMIT/TEST/DR1/FJEM0/SX4_rif.wav
Frame Level Accuracy:  0.37174721189591076 %

Test File:  ./TIMIT/TES

Frame Level Accuracy:  0.4016064257028112 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI2114_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX404_rif.wav
Frame Level Accuracy:  4.761904761904762 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA2_rif.wav
Frame Level Accuracy:  2.0408163265306123 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA1_rif.wav
Frame Level Accuracy:  7.086614173228346 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX44_rif.wav
Frame Level Accuracy:  2.82258064516129 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX224_rif.wav
Frame Level Accuracy:  2.380952380952381 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI854_rif.wav
Frame Level Accuracy:  0.9090909090909091 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX134_rif.wav
Frame Level Accuracy:  2.904564315352697 %

Test File:  ./TIMIT/TEST/DR8/MDAW1/SX13_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR8/MDAW1/SX103_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR8/MDAW1/SI1453_rif.wav
Frame Level Accuracy:

In [7]:
#Test with GMM with 4 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/004"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [8]:
print("Testing with GMM model with 4 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 4 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 4 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  2.1551724137931036 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  0.411522633744856 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  0.6535947712418301 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  0.37453183520599254 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.2976190476190476 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  1.2048192771084338 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  3.3816425120772946 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  1.9169329073482428 %

Test File:  ./TIMIT/TEST/DR4/MTLS0/SI2000_rif.wav
Fram

Frame Level Accuracy:  2.6615969581749046 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  1.015228426395939 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  0.8064516129032258 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  1.0 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  5.3061224489795915 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  2.8169014084507045 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  2.5236593059936907 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  0.3115264797507788 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  2.9661016949152543 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  4.054054054054054 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FADG0/SI1279_rif.wav


Frame Level Accuracy:  2.3255813953488373 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  4.8169556840077075 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  2.3178807947019866 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  0.6756756756756757 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  5.215827338129497 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  2.6578073089700998 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  1.2944983818770228 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  5.851063829787234 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  0.2985074626865672 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX94_rif.wav
Frame Level Accuracy:  1.6722408026755853 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1714_rif.wav
Frame Level Accuracy:  1.3559322033898304 %

Test File:  ./TIMIT/T

Frame Level Accuracy:  2.9850746268656714 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  11.76470588235294 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  1.2552301255230125 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  4.562737642585551 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  2.0761245674740483 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  2.0710059171597637 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  3.488372093023256 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI2027_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX137_rif.wav
Frame Level Accuracy:  0.6993006993006993 %

Test File:  ./TIMIT/TEST/DR4/FSEM0/SX298_rif.wav
Frame Level Accur

Frame Level Accuracy:  1.3729977116704806 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  0.4830917874396135 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  1.8518518518518516 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  2.1929824561403506 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  0.9259259259259258 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  0.3389830508474576 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  0.5633802816901409 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX383_rif.wav
Frame Level Accuracy:  2.2304832713754648 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX293_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX23_rif.wav
Frame Level Accuracy:  1.3824884792626728 %

Test File:  ./TIMIT/TEST/DR3/MJB

...

Frame Level Accuracy:  0.6269592476489028 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI929_rif.wav
Frame Level Accuracy:  5.737704918032787 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX299_rif.wav
Frame Level Accuracy:  0.7194244604316548 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI2189_rif.wav
Frame Level Accuracy:  2.030456852791878 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX29_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX389_rif.wav
Frame Level Accuracy:  7.5829383886255926 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA2_rif.wav
Frame Level Accuracy:  3.896103896103896 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA1_rif.wav
Frame Level Accuracy:  7.169811320754717 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI1559_rif.wav
Frame Level Accuracy:  3.125 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX209_rif.wav
Frame Level Accuracy:  1.7857142857142856 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX119_rif.wav
Frame Level Accuracy:  3.4482758620689653 %

Test File:  ./TIMIT/TEST/DR7/MGRT0/

Frame Level Accuracy:  0.3968253968253968 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX320_rif.wav
Frame Level Accuracy:  3.0395136778115504 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX313_rif.wav
Frame Level Accuracy:  2.608695652173913 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI1033_rif.wav
Frame Level Accuracy:  1.8867924528301887 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX403_rif.wav
Frame Level Accuracy:  2.4096385542168677 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA2_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA1_rif.wav
Frame Level Accuracy:  1.465201465201465 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX223_rif.wav
Frame Level Accuracy:  4.371584699453552 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX43_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI2293_rif.wav
Frame Level Accuracy:  0.6493506493506493 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI1663_rif.wav
Frame Level Accuracy:  1.8957345971563981 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX133_rif.wav
F

Frame Level Accuracy:  3.717472118959108 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX12_rif.wav
Frame Level Accuracy:  0.5263157894736842 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX102_rif.wav
Frame Level Accuracy:  1.8867924528301887 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1812_rif.wav
Frame Level Accuracy:  0.7722007722007722 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX372_rif.wav
Frame Level Accuracy:  0.3676470588235294 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1182_rif.wav
Frame Level Accuracy:  1.6736401673640167 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA2_rif.wav
Frame Level Accuracy:  2.5454545454545454 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI2313_rif.wav
Frame Level Accuracy:  1.8255578093306288 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA1_rif.wav
Frame Level Accuracy:  1.6233766233766231 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX192_rif.wav
Frame Level Accuracy:  3.2520325203252036 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX282_rif.wav
Frame Level Accuracy:  1.3937282229965158 %

Test File:  ./TIMI

Frame Level Accuracy:  0.5263157894736842 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI844_rif.wav
Frame Level Accuracy:  0.8941877794336811 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX110_rif.wav
Frame Level Accuracy:  5.668016194331984 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1640_rif.wav
Frame Level Accuracy:  1.3888888888888888 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA2_rif.wav
Frame Level Accuracy:  3.864734299516908 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA1_rif.wav
Frame Level Accuracy:  4.395604395604396 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX200_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI2270_rif.wav
Frame Level Accuracy:  6.572769953051644 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1010_rif.wav
Frame Level Accuracy:  1.7326732673267329 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX380_rif.wav
Frame Level Accuracy:  1.0582010582010581 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX290_rif.wav
Frame Level Accuracy:  1.2779552715654952 %

Test File:  ./TIMIT/

Frame Level Accuracy:  0.4608294930875576 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX137_rif.wav
Frame Level Accuracy:  0.8928571428571428 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX314_rif.wav
Frame Level Accuracy:  4.044117647058823 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI1484_rif.wav
Frame Level Accuracy:  0.8032128514056224 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI2114_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX404_rif.wav
Frame Level Accuracy:  1.9047619047619049 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA2_rif.wav
Frame Level Accuracy:  1.2244897959183674 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA1_rif.wav
Frame Level Accuracy:  4.724409448818897 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX44_rif.wav
Frame Level Accuracy:  2.0161290322580645 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX224_rif.wav
Frame Level Accuracy:  4.761904761904762 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI854_rif.wav
Frame Level Accuracy:  0.45454545454545453 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/

In [9]:
#Test with GMM with 8 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/008"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [10]:
print("Testing with GMM model with 8 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 8 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 8 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  0.8620689655172413 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  0.823045267489712 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  0.6535947712418301 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  1.4981273408239701 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  4.417670682730924 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  2.898550724637681 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  0.33670033670033667 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  1.9169329073482428 %

Test File:  ./TIMIT/TEST/DR4/MTLS0/SI2000_rif.wav
Frame 

Frame Level Accuracy:  2.2813688212927756 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  1.5228426395939088 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  0.8064516129032258 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  0.5 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  4.8979591836734695 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  1.056338028169014 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  4.416403785488959 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  0.3115264797507788 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  2.5423728813559325 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  2.027027027027027 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FADG0/SI1279_rif.wav
F

Frame Level Accuracy:  1.488095238095238 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  1.8604651162790697 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  3.275529865125241 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  3.642384105960265 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  1.3513513513513513 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  6.294964028776978 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  3.6544850498338874 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  2.26537216828479 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  6.382978723404255 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  0.5970149253731344 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX94_rif.wav
Frame Level Accuracy:  1.0033444816053512 %

Test File:  ./TIMIT/TEST/DR

Frame Level Accuracy:  3.2467532467532463 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  2.6119402985074625 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  7.9831932773109235 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  3.4220532319391634 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  0.34602076124567477 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  1.183431952662722 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  1.4534883720930232 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame Level Accuracy:  1.5151515151515151 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI2027_rif.wav
Frame Level Accuracy:  2.2801302931596092 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX137_rif.wav

Frame Level Accuracy:  2.0066889632107023 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  2.9748283752860414 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  2.631578947368421 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  0.9259259259259258 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  3.389830508474576 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  1.4084507042253522 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX383_rif.wav
Frame Level Accuracy:  2.973977695167286 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX293_rif.wav
Frame Level Accuracy:  0.38022813688212925 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX23_rif.wav
F

...

Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX408_rif.wav
Frame Level Accuracy:  4.032258064516129 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX318_rif.wav
Frame Level Accuracy:  6.425702811244979 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1488_rif.wav
Frame Level Accuracy:  2.507836990595611 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI929_rif.wav
Frame Level Accuracy:  2.8688524590163933 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX299_rif.wav
Frame Level Accuracy:  4.316546762589928 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI2189_rif.wav
Frame Level Accuracy:  2.5380710659898478 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX29_rif.wav
Frame Level Accuracy:  3.7037037037037033 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX389_rif.wav
Frame Level Accuracy:  6.161137440758294 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA2_rif.wav
Frame Level Accuracy:  3.463203463203463 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA1_rif.wav
Frame Level Accuracy:  4.150943396226415 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI155

Frame Level Accuracy:  1.7006802721088436 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX230_rif.wav
Frame Level Accuracy:  0.30959752321981426 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI950_rif.wav
Frame Level Accuracy:  1.0330578512396695 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX140_rif.wav
Frame Level Accuracy:  0.3968253968253968 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX320_rif.wav
Frame Level Accuracy:  1.21580547112462 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX313_rif.wav
Frame Level Accuracy:  3.4782608695652173 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI1033_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX403_rif.wav
Frame Level Accuracy:  0.4016064257028112 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA2_rif.wav
Frame Level Accuracy:  1.0204081632653061 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA1_rif.wav
Frame Level Accuracy:  2.197802197802198 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX223_rif.wav
Frame Level Accuracy:  2.185792349726776 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/S

Frame Level Accuracy:  0.6920415224913495 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SA1_rif.wav
Frame Level Accuracy:  6.315789473684211 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX383_rif.wav
Frame Level Accuracy:  1.2396694214876034 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX293_rif.wav
Frame Level Accuracy:  3.571428571428571 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX23_rif.wav
Frame Level Accuracy:  5.204460966542751 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX12_rif.wav
Frame Level Accuracy:  0.5263157894736842 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX102_rif.wav
Frame Level Accuracy:  1.8867924528301887 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1812_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX372_rif.wav
Frame Level Accuracy:  1.8382352941176472 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1182_rif.wav
Frame Level Accuracy:  2.928870292887029 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA2_rif.wav
Frame Level Accuracy:  3.6363636363636362 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI2

Frame Level Accuracy:  1.1235955056179776 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX124_rif.wav
Frame Level Accuracy:  1.2295081967213115 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI1474_rif.wav
Frame Level Accuracy:  0.7444168734491315 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX34_rif.wav
Frame Level Accuracy:  0.2631578947368421 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI844_rif.wav
Frame Level Accuracy:  0.7451564828614009 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX110_rif.wav
Frame Level Accuracy:  4.8582995951417 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1640_rif.wav
Frame Level Accuracy:  5.555555555555555 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA2_rif.wav
Frame Level Accuracy:  8.212560386473431 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA1_rif.wav
Frame Level Accuracy:  3.296703296703297 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX200_rif.wav
Frame Level Accuracy:  1.8518518518518516 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI2270_rif.wav
Frame Level Accuracy:  5.164319248826291 %

Test File:  ./TIMIT/TEST

Frame Level Accuracy:  3.7142857142857144 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX47_rif.wav
Frame Level Accuracy:  2.3255813953488373 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX227_rif.wav
Frame Level Accuracy:  0.5376344086021506 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI1217_rif.wav
Frame Level Accuracy:  2.1881838074398248 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI1847_rif.wav
Frame Level Accuracy:  0.4608294930875576 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX137_rif.wav
Frame Level Accuracy:  1.1904761904761905 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX314_rif.wav
Frame Level Accuracy:  4.044117647058823 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI1484_rif.wav
Frame Level Accuracy:  1.2048192771084338 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI2114_rif.wav
Frame Level Accuracy:  4.824561403508771 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX404_rif.wav
Frame Level Accuracy:  3.3333333333333335 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA2_rif.wav
Frame Level Accuracy:  4.8979591836734695 %

Test File:  ./TI

In [13]:
#Test with GMM with 16 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/016"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [12]:
print("Testing with GMM model with 16 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 16 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 16 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  1.7241379310344827 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  0.411522633744856 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  1.8726591760299627 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.2976190476190476 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  0.6756756756756757 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  0.4016064257028112 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  2.4154589371980677 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  3.0303030303030303 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  1.9169329073482428 %

Test File:  ./TIMIT/TEST/DR4/MTLS0/SI20

Frame Level Accuracy:  1.1406844106463878 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  5.83756345177665 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  1.2096774193548387 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  1.5 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  4.081632653061225 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  2.8169014084507045 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  4.100946372239748 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  1.2461059190031152 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  11.440677966101696 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  3.8288288288288284 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283_rif.wav
Frame Level Accuracy:  1.1940298507462688 %

Test File:  ./TIMIT/TEST/DR4/FADG0/SI

Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  2.697495183044316 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  1.3245033112582782 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  3.597122302158273 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  4.651162790697675 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  4.53074433656958 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  4.25531914893617 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  2.0895522388059704 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX94_rif.wav
Frame Level Accuracy:  0.33444816053511706 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1714_rif.wav
Frame Level Accuracy:  0.6779661016949152 %

Test File:  ./TIMIT/TEST/DR4/FGJD0/SX99

Frame Level Accuracy:  2.5974025974025974 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  2.2388059701492535 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  6.722689075630252 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  1.6736401673640167 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  4.182509505703422 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  2.0761245674740483 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  1.4792899408284024 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  2.616279069767442 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame Level Accuracy:  1.5151515151515151 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI2027_rif.wav
Frame Level Accuracy:  1.3029315960912053 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/S

Frame Level Accuracy:  1.3377926421404682 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  2.745995423340961 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  0.9259259259259258 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  2.631578947368421 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  1.8518518518518516 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  1.694915254237288 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  1.6901408450704223 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX383_rif.wav
Frame Level Accuracy:  5.947955390334572 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX293_rif.wav
Frame Level Accuracy:  0.38022813688212925 %

Test File:  ./TIMIT/TEST/DR3/MCTW0

...

Frame Level Accuracy:  4.417670682730924 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1488_rif.wav
Frame Level Accuracy:  1.5673981191222568 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI929_rif.wav
Frame Level Accuracy:  4.918032786885246 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX299_rif.wav
Frame Level Accuracy:  1.4388489208633095 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI2189_rif.wav
Frame Level Accuracy:  1.5228426395939088 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX29_rif.wav
Frame Level Accuracy:  1.3888888888888888 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX389_rif.wav
Frame Level Accuracy:  2.843601895734597 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA2_rif.wav
Frame Level Accuracy:  3.463203463203463 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SA1_rif.wav
Frame Level Accuracy:  2.6415094339622645 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI1559_rif.wav
Frame Level Accuracy:  1.0416666666666665 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX209_rif.wav
Frame Level Accuracy:  0.4464285714285714 %

Test File:  ./TIMIT/T

Frame Level Accuracy:  2.066115702479339 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX140_rif.wav
Frame Level Accuracy:  1.1904761904761905 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX320_rif.wav
Frame Level Accuracy:  1.5197568389057752 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX313_rif.wav
Frame Level Accuracy:  1.3043478260869565 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI1033_rif.wav
Frame Level Accuracy:  1.257861635220126 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX403_rif.wav
Frame Level Accuracy:  0.8032128514056224 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA2_rif.wav
Frame Level Accuracy:  1.530612244897959 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SA1_rif.wav
Frame Level Accuracy:  2.197802197802198 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX223_rif.wav
Frame Level Accuracy:  2.185792349726776 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX43_rif.wav
Frame Level Accuracy:  1.2048192771084338 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI2293_rif.wav
Frame Level Accuracy:  1.6233766233766231 %

Test File:  ./TIMIT/TES

Frame Level Accuracy:  1.6528925619834711 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX293_rif.wav
Frame Level Accuracy:  0.7936507936507936 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX23_rif.wav
Frame Level Accuracy:  1.1152416356877324 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX12_rif.wav
Frame Level Accuracy:  1.0526315789473684 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX102_rif.wav
Frame Level Accuracy:  1.257861635220126 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1812_rif.wav
Frame Level Accuracy:  1.1583011583011582 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX372_rif.wav
Frame Level Accuracy:  3.6764705882352944 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI1182_rif.wav
Frame Level Accuracy:  1.6736401673640167 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA2_rif.wav
Frame Level Accuracy:  3.6363636363636362 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SI2313_rif.wav
Frame Level Accuracy:  1.4198782961460445 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SA1_rif.wav
Frame Level Accuracy:  1.948051948051948 %

Test File:  ./TIMIT/

Frame Level Accuracy:  0.24813895781637718 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX34_rif.wav
Frame Level Accuracy:  0.5263157894736842 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI844_rif.wav
Frame Level Accuracy:  2.086438152011923 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX110_rif.wav
Frame Level Accuracy:  4.8582995951417 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1640_rif.wav
Frame Level Accuracy:  4.166666666666666 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA2_rif.wav
Frame Level Accuracy:  4.830917874396135 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SA1_rif.wav
Frame Level Accuracy:  8.424908424908425 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX200_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI2270_rif.wav
Frame Level Accuracy:  8.92018779342723 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI1010_rif.wav
Frame Level Accuracy:  1.2376237623762376 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX380_rif.wav
Frame Level Accuracy:  2.1164021164021163 %

Test File:  ./TIMIT/TEST/

Frame Level Accuracy:  1.312910284463895 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI1847_rif.wav
Frame Level Accuracy:  6.912442396313365 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX137_rif.wav
Frame Level Accuracy:  0.2976190476190476 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX314_rif.wav
Frame Level Accuracy:  2.2058823529411766 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI1484_rif.wav
Frame Level Accuracy:  1.6064257028112447 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SI2114_rif.wav
Frame Level Accuracy:  2.1929824561403506 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX404_rif.wav
Frame Level Accuracy:  1.4285714285714286 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA2_rif.wav
Frame Level Accuracy:  3.6734693877551026 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SA1_rif.wav
Frame Level Accuracy:  7.086614173228346 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX44_rif.wav
Frame Level Accuracy:  0.4032258064516129 %

Test File:  ./TIMIT/TEST/DR8/FJSJ0/SX224_rif.wav
Frame Level Accuracy:  0.5952380952380952 %

Test File:  ./TIMIT/

In [14]:
#Test with GMM with 32 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/032"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [15]:
print("Testing with GMM model with 32 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 32 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 32 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  2.1551724137931036 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  0.823045267489712 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  1.3071895424836601 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  4.119850187265917 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.5952380952380952 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  3.3783783783783785 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  4.819277108433735 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  4.830917874396135 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  2.356902356902357 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  2.8753993610223643 %

Test File:  ./TIMIT/TEST/DR4

Frame Level Accuracy:  3.3333333333333335 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SI1088_rif.wav
Frame Level Accuracy:  0.7604562737642585 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  2.284263959390863 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  1.0 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  1.2244897959183674 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  3.169014084507042 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  4.73186119873817 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  0.6230529595015576 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  6.779661016949152 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  3.8288288288288284 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283_rif.wav
Fra

Frame Level Accuracy:  4.761904761904762 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SI1502_rif.wav
Frame Level Accuracy:  2.976190476190476 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  2.3255813953488373 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  2.119460500963391 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  3.642384105960265 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  3.41726618705036 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  1.6611295681063125 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  2.5889967637540456 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  4.25531914893617 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  2.9850746268656714 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX94_r

Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX140_rif.wav
Frame Level Accuracy:  1.893939393939394 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SI1850_rif.wav
Frame Level Accuracy:  2.5974025974025974 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  2.2388059701492535 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  6.302521008403361 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  1.520912547528517 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  1.384083044982699 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  2.366863905325444 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  0.8064516129032258 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  1.744186046511628 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX227_rif.wav
Frame

Frame Level Accuracy:  4.580152671755725 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI2179_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX109_rif.wav
Frame Level Accuracy:  0.6688963210702341 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  4.805491990846682 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  1.4492753623188406 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  0.46511627906976744 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  1.3888888888888888 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  1.7543859649122806 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  4.62962962962963 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  3.389830508474576 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI1373_rif.wav
Frame Level Accuracy:  2.2535211267605635 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/

...

Frame Level Accuracy:  3.8860103626943006 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1026_rif.wav
Frame Level Accuracy:  7.6923076923076925 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA2_rif.wav
Frame Level Accuracy:  3.65296803652968 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX48_rif.wav
Frame Level Accuracy:  2.0 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA1_rif.wav
Frame Level Accuracy:  7.4324324324324325 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX228_rif.wav
Frame Level Accuracy:  1.1428571428571428 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX408_rif.wav
Frame Level Accuracy:  4.435483870967742 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX318_rif.wav
Frame Level Accuracy:  1.6064257028112447 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1488_rif.wav
Frame Level Accuracy:  0.6269592476489028 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI929_rif.wav
Frame Level Accuracy:  4.918032786885246 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SX299_rif.wav
Frame Level Accuracy:  3.9568345323741005 %

Test File:  ./TIMIT/TEST/DR7/FDHC0/SI

Frame Level Accuracy:  3.932584269662921 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI1580_rif.wav
Frame Level Accuracy:  1.9011406844106464 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX410_rif.wav
Frame Level Accuracy:  2.4691358024691357 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA2_rif.wav
Frame Level Accuracy:  2.877697841726619 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA1_rif.wav
Frame Level Accuracy:  1.5105740181268883 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX50_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX230_rif.wav
Frame Level Accuracy:  0.6191950464396285 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI950_rif.wav
Frame Level Accuracy:  4.132231404958678 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX140_rif.wav
Frame Level Accuracy:  0.7936507936507936 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX320_rif.wav
Frame Level Accuracy:  4.5592705167173255 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SX313_rif.wav
Frame Level Accuracy:  1.3043478260869565 %

Test File:  ./TIMIT/TEST/DR6/MJFC0/SI

Frame Level Accuracy:  1.4981273408239701 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX113_rif.wav
Frame Level Accuracy:  2.710027100271003 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1283_rif.wav
Frame Level Accuracy:  2.914798206278027 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1423_rif.wav
Frame Level Accuracy:  0.4629629629629629 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SA2_rif.wav
Frame Level Accuracy:  4.484304932735426 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX203_rif.wav
Frame Level Accuracy:  3.4602076124567476 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SA1_rif.wav
Frame Level Accuracy:  3.1578947368421053 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX383_rif.wav
Frame Level Accuracy:  1.6528925619834711 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX293_rif.wav
Frame Level Accuracy:  1.5873015873015872 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX23_rif.wav
Frame Level Accuracy:  1.858736059479554 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX12_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR6/MRJR0/SX1

Frame Level Accuracy:  4.1558441558441555 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI2104_rif.wav
Frame Level Accuracy:  1.1709601873536302 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA2_rif.wav
Frame Level Accuracy:  4.427083333333334 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA1_rif.wav
Frame Level Accuracy:  6.762295081967213 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX214_rif.wav
Frame Level Accuracy:  2.518891687657431 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX394_rif.wav
Frame Level Accuracy:  1.9662921348314606 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX124_rif.wav
Frame Level Accuracy:  4.918032786885246 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI1474_rif.wav
Frame Level Accuracy:  0.0 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX34_rif.wav
Frame Level Accuracy:  1.0526315789473684 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI844_rif.wav
Frame Level Accuracy:  2.533532041728763 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SX110_rif.wav
Frame Level Accuracy:  2.834008097165992 %

Test File:  ./TIMIT/TEST/DR1/MJSW0/SI16

Frame Level Accuracy:  0.9615384615384616 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX317_rif.wav
Frame Level Accuracy:  2.3255813953488373 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI587_rif.wav
Frame Level Accuracy:  3.4602076124567476 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SA2_rif.wav
Frame Level Accuracy:  5.5045871559633035 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX407_rif.wav
Frame Level Accuracy:  2.727272727272727 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SA1_rif.wav
Frame Level Accuracy:  5.714285714285714 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX47_rif.wav
Frame Level Accuracy:  1.3953488372093024 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX227_rif.wav
Frame Level Accuracy:  2.1505376344086025 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI1217_rif.wav
Frame Level Accuracy:  1.5317286652078774 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI1847_rif.wav
Frame Level Accuracy:  5.529953917050691 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX137_rif.wav
Frame Level Accuracy:  1.7857142857142856 %

Test File:  ./TIMIT/T

In [16]:
#Test with GMM with 128 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/128"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [17]:
print("Testing with GMM model with 128 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 128 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 128 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  1.293103448275862 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  3.292181069958848 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  2.6143790849673203 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  4.119850187265917 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  0.2976190476190476 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  3.3783783783783785 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  4.819277108433735 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  5.797101449275362 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  2.0202020202020203 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  4.15335463258786 %

Test File:  ./TIMIT/TEST/DR4/

Frame Level Accuracy:  3.3333333333333335 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SI1088_rif.wav
Frame Level Accuracy:  4.182509505703422 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  4.314720812182741 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  4.838709677419355 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  4.0 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  2.0408163265306123 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  2.464788732394366 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  2.5236593059936907 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  4.049844236760125 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  4.23728813559322 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  4.054054054054054 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX283

Frame Level Accuracy:  1.9047619047619049 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SI1502_rif.wav
Frame Level Accuracy:  3.273809523809524 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  4.651162790697675 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  1.9267822736030826 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  2.6490066225165565 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  2.7027027027027026 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  3.0575539568345325 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  1.3289036544850499 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  4.53074433656958 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  6.382978723404255 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  8.358208955223882 %

Test File:  ./TIMIT/TEST

Frame Level Accuracy:  1.3986013986013985 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX140_rif.wav
Frame Level Accuracy:  2.272727272727273 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SI1850_rif.wav
Frame Level Accuracy:  3.2467532467532463 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  2.9850746268656714 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  5.88235294117647 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  3.765690376569038 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  1.9011406844106464 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  1.7301038062283738 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  1.4792899408284024 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  1.2096774193548387 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  3.77906976744186 %

Test File:  ./TIMIT/TEST

Frame Level Accuracy:  0.546448087431694 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX19_rif.wav
Frame Level Accuracy:  1.5267175572519083 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI2179_rif.wav
Frame Level Accuracy:  6.467661691542288 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX109_rif.wav
Frame Level Accuracy:  5.016722408026756 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  3.6613272311212817 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  1.4492753623188406 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  1.8604651162790697 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  10.185185185185185 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  4.385964912280701 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  2.7777777777777777 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  2.0338983050847457 %

Test File:  ./TIMIT/TE

...

Frame Level Accuracy:  2.564102564102564 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SA1_rif.wav
Frame Level Accuracy:  5.405405405405405 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX193_rif.wav
Frame Level Accuracy:  1.4851485148514851 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SI1273_rif.wav
Frame Level Accuracy:  3.5623409669211195 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX283_rif.wav
Frame Level Accuracy:  1.3605442176870748 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX138_rif.wav
Frame Level Accuracy:  5.2434456928838955 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI858_rif.wav
Frame Level Accuracy:  4.922279792746114 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1026_rif.wav
Frame Level Accuracy:  4.733727810650888 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA2_rif.wav
Frame Level Accuracy:  4.5662100456621 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX48_rif.wav
Frame Level Accuracy:  4.3999999999999995 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA1_rif.wav
Frame Level Accuracy:  14.527027027027026 %

Test File:  ./TIMIT/TEST/DR

Frame Level Accuracy:  2.8301886792452833 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX396_rif.wav
Frame Level Accuracy:  2.601156069364162 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI2196_rif.wav
Frame Level Accuracy:  2.9023746701846966 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI936_rif.wav
Frame Level Accuracy:  3.7735849056603774 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX126_rif.wav
Frame Level Accuracy:  3.0837004405286343 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX36_rif.wav
Frame Level Accuracy:  5.111821086261981 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI2210_rif.wav
Frame Level Accuracy:  4.49438202247191 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI1580_rif.wav
Frame Level Accuracy:  0.7604562737642585 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX410_rif.wav
Frame Level Accuracy:  7.4074074074074066 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA2_rif.wav
Frame Level Accuracy:  3.237410071942446 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA1_rif.wav
Frame Level Accuracy:  2.1148036253776437 %

Test File:  ./TIMIT/TE

Frame Level Accuracy:  7.633587786259542 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX394_rif.wav
Frame Level Accuracy:  3.3333333333333335 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI1564_rif.wav
Frame Level Accuracy:  1.2953367875647668 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX124_rif.wav
Frame Level Accuracy:  2.955665024630542 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI934_rif.wav
Frame Level Accuracy:  0.6514657980456027 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI2194_rif.wav
Frame Level Accuracy:  5.048076923076923 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX34_rif.wav
Frame Level Accuracy:  2.898550724637681 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI653_rif.wav
Frame Level Accuracy:  2.247191011235955 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX113_rif.wav
Frame Level Accuracy:  2.168021680216802 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1283_rif.wav
Frame Level Accuracy:  2.690582959641256 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1423_rif.wav
Frame Level Accuracy:  4.166666666666666 %

Test File:  ./TIMIT/

Frame Level Accuracy:  3.5416666666666665 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX216_rif.wav
Frame Level Accuracy:  7.853403141361256 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA2_rif.wav
Frame Level Accuracy:  0.390625 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SI756_rif.wav
Frame Level Accuracy:  2.933333333333333 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA1_rif.wav
Frame Level Accuracy:  3.5398230088495577 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX396_rif.wav
Frame Level Accuracy:  1.8072289156626504 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX126_rif.wav
Frame Level Accuracy:  3.041825095057034 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX36_rif.wav
Frame Level Accuracy:  3.8461538461538463 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX304_rif.wav
Frame Level Accuracy:  2.0779220779220777 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI2104_rif.wav
Frame Level Accuracy:  2.576112412177986 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA2_rif.wav
Frame Level Accuracy:  5.729166666666666 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/S

Frame Level Accuracy:  4.229607250755287 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX189_rif.wav
Frame Level Accuracy:  6.94980694980695 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI819_rif.wav
Frame Level Accuracy:  2.5316455696202533 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI2079_rif.wav
Frame Level Accuracy:  8.641975308641975 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX9_rif.wav
Frame Level Accuracy:  0.8771929824561403 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX369_rif.wav
Frame Level Accuracy:  6.153846153846154 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI1449_rif.wav
Frame Level Accuracy:  4.557640750670242 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX279_rif.wav
Frame Level Accuracy:  2.564102564102564 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX317_rif.wav
Frame Level Accuracy:  5.5813953488372094 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI587_rif.wav
Frame Level Accuracy:  3.4602076124567476 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SA2_rif.wav
Frame Level Accuracy:  4.128440366972478 %

Test File:  ./TIMIT/TEST/

In [18]:
#Test with GMM with 256 mixtures
modelpath = "./models/MFCC_wo_energy_coeff/256"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [19]:
print("Testing with GMM model with 256 mixtures")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 256 mixtures: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 256 mixtures
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  3.0172413793103448 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  3.292181069958848 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  2.6143790849673203 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  4.868913857677903 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  1.1904761904761905 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  1.6891891891891893 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  7.228915662650602 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  4.3478260869565215 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  3.0303030303030303 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  4.15335463258786 %

Test File:  ./TIMIT/TEST/DR

Frame Level Accuracy:  3.6363636363636362 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SI1088_rif.wav
Frame Level Accuracy:  4.562737642585551 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  3.2994923857868024 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  4.838709677419355 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  3.5000000000000004 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  1.6326530612244898 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  2.112676056338028 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  3.7854889589905363 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  4.361370716510903 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  3.389830508474576 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav
Frame Level Accuracy:  3.3783783783783785 %

Test File:  ./TIMIT/TE

Frame Level Accuracy:  3.8095238095238098 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SI1502_rif.wav
Frame Level Accuracy:  1.7857142857142856 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  1.3953488372093024 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  5.5876685934489405 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  2.6490066225165565 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  2.027027027027027 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  4.496402877697841 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  1.9933554817275747 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  3.2362459546925564 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  3.9893617021276597 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX184_rif.wav
Frame Level Accuracy:  3.582089552238806 %

Test File:  ./TIMIT/T

Frame Level Accuracy:  1.3986013986013985 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX140_rif.wav
Frame Level Accuracy:  2.272727272727273 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SI1850_rif.wav
Frame Level Accuracy:  6.493506493506493 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  3.731343283582089 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  4.621848739495799 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  4.602510460251046 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  2.6615969581749046 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  3.8062283737024223 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  2.0710059171597637 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  2.0161290322580645 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI767_rif.wav
Frame Level Accuracy:  1.4534883720930232 %

Test File:  ./TIMIT/TES

Frame Level Accuracy:  1.092896174863388 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX19_rif.wav
Frame Level Accuracy:  0.7633587786259541 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI2179_rif.wav
Frame Level Accuracy:  3.482587064676617 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX109_rif.wav
Frame Level Accuracy:  0.6688963210702341 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  2.2883295194508007 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  2.898550724637681 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  1.3953488372093024 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  4.166666666666666 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  3.9473684210526314 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  2.7777777777777777 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA1_rif.wav
Frame Level Accuracy:  3.389830508474576 %

Test File:  ./TIMIT/TES

...

Frame Level Accuracy:  2.9914529914529915 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SA1_rif.wav
Frame Level Accuracy:  4.054054054054054 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX193_rif.wav
Frame Level Accuracy:  3.4653465346534658 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SI1273_rif.wav
Frame Level Accuracy:  4.580152671755725 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX283_rif.wav
Frame Level Accuracy:  1.0204081632653061 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX138_rif.wav
Frame Level Accuracy:  3.7453183520599254 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI858_rif.wav
Frame Level Accuracy:  6.994818652849741 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1026_rif.wav
Frame Level Accuracy:  1.7751479289940828 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA2_rif.wav
Frame Level Accuracy:  3.1963470319634704 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX48_rif.wav
Frame Level Accuracy:  9.6 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA1_rif.wav
Frame Level Accuracy:  8.783783783783784 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX22

Frame Level Accuracy:  3.1446540880503147 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX396_rif.wav
Frame Level Accuracy:  2.8901734104046244 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI2196_rif.wav
Frame Level Accuracy:  3.6939313984168867 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI936_rif.wav
Frame Level Accuracy:  3.7735849056603774 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX126_rif.wav
Frame Level Accuracy:  3.0837004405286343 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX36_rif.wav
Frame Level Accuracy:  5.431309904153355 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI2210_rif.wav
Frame Level Accuracy:  1.6853932584269662 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI1580_rif.wav
Frame Level Accuracy:  3.041825095057034 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX410_rif.wav
Frame Level Accuracy:  5.349794238683128 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA2_rif.wav
Frame Level Accuracy:  2.158273381294964 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA1_rif.wav
Frame Level Accuracy:  1.5105740181268883 %

Test File:  ./TIMIT/T

Frame Level Accuracy:  4.961832061068702 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX394_rif.wav
Frame Level Accuracy:  2.380952380952381 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI1564_rif.wav
Frame Level Accuracy:  3.3678756476683938 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX124_rif.wav
Frame Level Accuracy:  1.9704433497536946 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI934_rif.wav
Frame Level Accuracy:  2.2801302931596092 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI2194_rif.wav
Frame Level Accuracy:  4.326923076923077 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX34_rif.wav
Frame Level Accuracy:  0.966183574879227 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI653_rif.wav
Frame Level Accuracy:  5.992509363295881 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX113_rif.wav
Frame Level Accuracy:  3.523035230352303 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1283_rif.wav
Frame Level Accuracy:  4.484304932735426 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1423_rif.wav
Frame Level Accuracy:  5.092592592592593 %

Test File:  ./TIMIT/

Frame Level Accuracy:  2.7083333333333335 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX216_rif.wav
Frame Level Accuracy:  8.900523560209423 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA2_rif.wav
Frame Level Accuracy:  1.953125 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SI756_rif.wav
Frame Level Accuracy:  2.1333333333333333 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA1_rif.wav
Frame Level Accuracy:  4.424778761061947 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX396_rif.wav
Frame Level Accuracy:  4.216867469879518 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX126_rif.wav
Frame Level Accuracy:  4.562737642585551 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX36_rif.wav
Frame Level Accuracy:  3.4965034965034967 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX304_rif.wav
Frame Level Accuracy:  2.3376623376623376 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI2104_rif.wav
Frame Level Accuracy:  4.215456674473068 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SA2_rif.wav
Frame Level Accuracy:  3.6458333333333335 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/S

Frame Level Accuracy:  4.273504273504273 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SA1_rif.wav
Frame Level Accuracy:  5.13595166163142 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX189_rif.wav
Frame Level Accuracy:  8.880308880308881 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI819_rif.wav
Frame Level Accuracy:  2.7848101265822782 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI2079_rif.wav
Frame Level Accuracy:  6.172839506172839 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX9_rif.wav
Frame Level Accuracy:  1.3157894736842104 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX369_rif.wav
Frame Level Accuracy:  6.923076923076923 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI1449_rif.wav
Frame Level Accuracy:  4.021447721179625 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX279_rif.wav
Frame Level Accuracy:  2.8846153846153846 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX317_rif.wav
Frame Level Accuracy:  3.7209302325581395 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SI587_rif.wav
Frame Level Accuracy:  2.0761245674740483 %

Test File:  ./TIMIT/TEST

In [20]:
#Test with GMM with 64 mixtures and mfcc features with energy coeff
modelpath = "./models/MFCC/064"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [21]:
print("Testing with GMM model with 64 mixtures and mfcc features with energy coeff")
avg_accuracy = 0
for full_wav in full_wavs:
    print("Test File: ", full_wav)

    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    accuracy = accuracy_score(mfcc_labels, pred) * 100
    print("Frame Level Accuracy: ", accuracy, "%\n")           
    avg_accuracy += accuracy
    
print("Accuracy for GMM with 64 mixtures and mfcc features with energy coeff: ", avg_accuracy/len(full_wavs), "%")
accuracy_list.append(avg_accuracy)

Testing with GMM model with 64 mixtures and mfcc features with energy coeff
Test File:  ./TIMIT/TEST/DR4/MGMM0/SX139_rif.wav
Frame Level Accuracy:  11.206896551724139 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA2_rif.wav
Frame Level Accuracy:  12.345679012345679 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX229_rif.wav
Frame Level Accuracy:  8.49673202614379 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SA1_rif.wav
Frame Level Accuracy:  32.20973782771536 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX49_rif.wav
Frame Level Accuracy:  13.988095238095239 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX409_rif.wav
Frame Level Accuracy:  7.4324324324324325 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1759_rif.wav
Frame Level Accuracy:  17.269076305220885 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI499_rif.wav
Frame Level Accuracy:  14.009661835748794 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SX319_rif.wav
Frame Level Accuracy:  9.090909090909092 %

Test File:  ./TIMIT/TEST/DR4/MGMM0/SI1129_rif.wav
Frame Level Accuracy:  17.2523961661

Frame Level Accuracy:  5.806451612903226 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SX278_rif.wav
Frame Level Accuracy:  10.0 %

Test File:  ./TIMIT/TEST/DR4/FCRH0/SI1088_rif.wav
Frame Level Accuracy:  8.745247148288973 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI733_rif.wav
Frame Level Accuracy:  20.304568527918782 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX13_rif.wav
Frame Level Accuracy:  10.080645161290322 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX103_rif.wav
Frame Level Accuracy:  20.5 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX373_rif.wav
Frame Level Accuracy:  15.10204081632653 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA2_rif.wav
Frame Level Accuracy:  26.408450704225352 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SA1_rif.wav
Frame Level Accuracy:  21.451104100946374 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SX193_rif.wav
Frame Level Accuracy:  17.75700934579439 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1993_rif.wav
Frame Level Accuracy:  21.610169491525426 %

Test File:  ./TIMIT/TEST/DR4/MLLL0/SI1363_rif.wav


Frame Level Accuracy:  8.558558558558559 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX408_rif.wav
Frame Level Accuracy:  14.761904761904763 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SI1502_rif.wav
Frame Level Accuracy:  10.416666666666668 %

Test File:  ./TIMIT/TEST/DR4/FDMS0/SX318_rif.wav
Frame Level Accuracy:  9.30232558139535 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1084_rif.wav
Frame Level Accuracy:  7.514450867052023 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX274_rif.wav
Frame Level Accuracy:  11.589403973509933 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SI1653_rif.wav
Frame Level Accuracy:  13.513513513513514 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX364_rif.wav
Frame Level Accuracy:  9.712230215827338 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SX4_rif.wav
Frame Level Accuracy:  10.299003322259136 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA2_rif.wav
Frame Level Accuracy:  20.711974110032365 %

Test File:  ./TIMIT/TEST/DR4/FEDW0/SA1_rif.wav
Frame Level Accuracy:  21.01063829787234 %

Test File:  ./TIMIT/TEST

Frame Level Accuracy:  15.96958174904943 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX230_rif.wav
Frame Level Accuracy:  8.041958041958042 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX140_rif.wav
Frame Level Accuracy:  15.151515151515152 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SI1850_rif.wav
Frame Level Accuracy:  12.337662337662337 %

Test File:  ./TIMIT/TEST/DR4/MBNS0/SX320_rif.wav
Frame Level Accuracy:  17.16417910447761 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX317_rif.wav
Frame Level Accuracy:  7.9831932773109235 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA2_rif.wav
Frame Level Accuracy:  20.502092050209207 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SI1397_rif.wav
Frame Level Accuracy:  9.125475285171103 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX407_rif.wav
Frame Level Accuracy:  10.380622837370241 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SA1_rif.wav
Frame Level Accuracy:  19.526627218934912 %

Test File:  ./TIMIT/TEST/DR4/MRKO0/SX47_rif.wav
Frame Level Accuracy:  6.451612903225806 %

Test File:  ./TIMIT/TES

Frame Level Accuracy:  9.282700421940929 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI1549_rif.wav
Frame Level Accuracy:  8.19672131147541 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX19_rif.wav
Frame Level Accuracy:  9.923664122137405 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI2179_rif.wav
Frame Level Accuracy:  7.462686567164178 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SX109_rif.wav
Frame Level Accuracy:  12.709030100334449 %

Test File:  ./TIMIT/TEST/DR3/MCSH0/SI919_rif.wav
Frame Level Accuracy:  5.949656750572083 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI2003_rif.wav
Frame Level Accuracy:  14.009661835748794 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX113_rif.wav
Frame Level Accuracy:  15.348837209302326 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SI743_rif.wav
Frame Level Accuracy:  16.666666666666664 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SA2_rif.wav
Frame Level Accuracy:  15.789473684210526 %

Test File:  ./TIMIT/TEST/DR3/MCTW0/SX203_rif.wav
Frame Level Accuracy:  7.4074074074074066 %

Test File:  ./TIMIT/T

...

Frame Level Accuracy:  10.67193675889328 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SA2_rif.wav
Frame Level Accuracy:  23.076923076923077 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SA1_rif.wav
Frame Level Accuracy:  29.72972972972973 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX193_rif.wav
Frame Level Accuracy:  5.445544554455446 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SI1273_rif.wav
Frame Level Accuracy:  9.669211195928753 %

Test File:  ./TIMIT/TEST/DR7/MKDR0/SX283_rif.wav
Frame Level Accuracy:  17.346938775510203 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX138_rif.wav
Frame Level Accuracy:  7.116104868913857 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI858_rif.wav
Frame Level Accuracy:  14.248704663212436 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SI1026_rif.wav
Frame Level Accuracy:  10.059171597633137 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA2_rif.wav
Frame Level Accuracy:  15.981735159817351 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SX48_rif.wav
Frame Level Accuracy:  10.4 %

Test File:  ./TIMIT/TEST/DR7/FLAS0/SA1_

Frame Level Accuracy:  16.50485436893204 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SA1_rif.wav
Frame Level Accuracy:  42.45283018867924 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX396_rif.wav
Frame Level Accuracy:  12.427745664739884 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI2196_rif.wav
Frame Level Accuracy:  12.66490765171504 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SI936_rif.wav
Frame Level Accuracy:  14.150943396226415 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX126_rif.wav
Frame Level Accuracy:  14.977973568281937 %

Test File:  ./TIMIT/TEST/DR7/MDVC0/SX36_rif.wav
Frame Level Accuracy:  14.376996805111823 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI2210_rif.wav
Frame Level Accuracy:  15.168539325842698 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SI1580_rif.wav
Frame Level Accuracy:  7.604562737642586 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SX410_rif.wav
Frame Level Accuracy:  15.22633744855967 %

Test File:  ./TIMIT/TEST/DR7/MNJM0/SA2_rif.wav
Frame Level Accuracy:  16.906474820143885 %

Test File:  ./TIMIT/TE

Frame Level Accuracy:  29.141104294478527 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX214_rif.wav
Frame Level Accuracy:  10.687022900763358 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX394_rif.wav
Frame Level Accuracy:  20.0 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI1564_rif.wav
Frame Level Accuracy:  9.067357512953368 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX124_rif.wav
Frame Level Accuracy:  15.270935960591133 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI934_rif.wav
Frame Level Accuracy:  9.77198697068404 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SI2194_rif.wav
Frame Level Accuracy:  14.663461538461538 %

Test File:  ./TIMIT/TEST/DR6/FMGD0/SX34_rif.wav
Frame Level Accuracy:  17.391304347826086 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI653_rif.wav
Frame Level Accuracy:  8.98876404494382 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SX113_rif.wav
Frame Level Accuracy:  10.02710027100271 %

Test File:  ./TIMIT/TEST/DR6/FDRW0/SI1283_rif.wav
Frame Level Accuracy:  8.520179372197308 %

Test File:  ./TIMIT/TEST/DR6/FDRW0

Frame Level Accuracy:  11.363636363636363 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SI1386_rif.wav
Frame Level Accuracy:  8.958333333333334 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX216_rif.wav
Frame Level Accuracy:  12.56544502617801 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA2_rif.wav
Frame Level Accuracy:  25.390625 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SI756_rif.wav
Frame Level Accuracy:  6.666666666666667 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SA1_rif.wav
Frame Level Accuracy:  26.843657817109147 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX396_rif.wav
Frame Level Accuracy:  10.240963855421686 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX126_rif.wav
Frame Level Accuracy:  12.167300380228136 %

Test File:  ./TIMIT/TEST/DR1/FELC0/SX36_rif.wav
Frame Level Accuracy:  10.48951048951049 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SX304_rif.wav
Frame Level Accuracy:  14.025974025974024 %

Test File:  ./TIMIT/TEST/DR1/FDAC1/SI2104_rif.wav
Frame Level Accuracy:  7.728337236533958 %

Test File:  ./TIMIT/TEST/DR1/FDA

Frame Level Accuracy:  11.151079136690647 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SA2_rif.wav
Frame Level Accuracy:  18.803418803418804 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SA1_rif.wav
Frame Level Accuracy:  24.169184290030213 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX189_rif.wav
Frame Level Accuracy:  9.266409266409266 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI819_rif.wav
Frame Level Accuracy:  13.164556962025317 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI2079_rif.wav
Frame Level Accuracy:  11.728395061728394 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX9_rif.wav
Frame Level Accuracy:  7.017543859649122 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX369_rif.wav
Frame Level Accuracy:  9.230769230769232 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SI1449_rif.wav
Frame Level Accuracy:  11.796246648793565 %

Test File:  ./TIMIT/TEST/DR8/MJLN0/SX279_rif.wav
Frame Level Accuracy:  6.41025641025641 %

Test File:  ./TIMIT/TEST/DR8/MRES0/SX317_rif.wav
Frame Level Accuracy:  10.69767441860465 %

Test File:  ./TIMIT/TEST/

In [27]:
print("Accuracy for GMM with 2 mixtures: ", accuracy_list[1]/len(full_wavs), "%")
print("Accuracy for GMM with 4 mixtures: ", accuracy_list[2]/len(full_wavs), "%")
print("Accuracy for GMM with 8 mixtures: ", accuracy_list[3]/len(full_wavs), "%")
print("Accuracy for GMM with 16 mixtures: ", accuracy_list[4]/len(full_wavs), "%")
print("Accuracy for GMM with 32 mixtures: ", accuracy_list[5]/len(full_wavs), "%")
print("Accuracy for GMM with 64 mixtures: ", accuracy_list[0]/len(full_wavs), "%")
print("Accuracy for GMM with 128 mixtures: ", accuracy_list[6]/len(full_wavs), "%")
print("Accuracy for GMM with 256 mixtures: ", accuracy_list[7]/len(full_wavs), "%")
print("Accuracy for GMM with 64 mixtures and features with energy coeff: ", accuracy_list[8]/len(full_wavs), "%")
print("Maximum Accuracy: ", np.max(accuracy_list)/len(full_wavs), "%")

Accuracy for GMM with 2 mixtures:  2.520850162933916 %
Accuracy for GMM with 4 mixtures:  2.3258566525441404 %
Accuracy for GMM with 8 mixtures:  2.4912475829840477 %
Accuracy for GMM with 16 mixtures:  2.20157864981258 %
Accuracy for GMM with 32 mixtures:  2.7243270682726677 %
Accuracy for GMM with 64 mixtures:  2.281282773357212 %
Accuracy for GMM with 128 mixtures:  3.735437599524438 %
Accuracy for GMM with 256 mixtures:  3.8271399584078325 %
Accuracy for GMM with 64 mixtures and features with energy coeff:  14.235578344131028 %
Maximum Accuracy:  14.235578344131028 %


# Phoneme Error Rate

In [3]:
#Test with GMM with 64 mixtures and mfcc features with energy coeff
modelpath = "./models/MFCC/064"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [11]:
print("Testing with GMM model with 64 mixtures and mfcc features with energy coeff")

pred_file = open("./pred/pred_64_ec.txt","w") 
label_file = open("./pred/label_64_ec.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_64_ec.txt\'")

Testing with GMM model with 64 mixtures and mfcc features with energy coeff
Predictions written to file 'pred_64_ec.txt'


In [14]:
print("PER for GMM Model with 64 mixtures and mfcc features with energy coefficient:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_64_ec.txt", "./pred/pred_64_ec.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 64 mixtures and mfcc features with energy coefficient:
[b'Sentence count: 1672', b'WER:    86.457% (    379334 /     438756)', b'WRR:    15.356% (     67375 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [15]:
#Test with GMM with 2 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/002"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [17]:
print("Testing with GMM model with 2 mixtures")

pred_file = open("./pred/pred_2.txt","w") 
label_file = open("./pred/label_2.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_2.txt\'")

Testing with GMM model with 2 mixtures
Predictions written to file 'pred_2.txt'


In [18]:
print("PER for GMM Model with 2 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_2.txt", "./pred/pred_2.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 2 mixtures:
[b'Sentence count: 1672', b'WER:    98.095% (    430396 /     438756)', b'WRR:     4.881% (     21417 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [19]:
#Test with GMM with 4 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/004"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [20]:
print("Testing with GMM model with 4 mixtures")

pred_file = open("./pred/pred_4.txt","w") 
label_file = open("./pred/label_4.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_4.txt\'")

Testing with GMM model with 4 mixtures
Predictions written to file 'pred_4.txt'


In [3]:
print("PER for GMM Model with 4 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_4.txt", "./pred/pred_4.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 4 mixtures:
[b'Sentence count: 1680', b'WER:    98.293% (    433292 /     440816)', b'WRR:     4.683% (     20645 /     440816)', b'SER:   100.000% (      1680 /       1680)']


In [4]:
#Test with GMM with 8 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/008"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [5]:
print("Testing with GMM model with 8 mixtures")

pred_file = open("./pred/pred_8.txt","w") 
label_file = open("./pred/label_8.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_8.txt\'")

Testing with GMM model with 8 mixtures
Predictions written to file 'pred_8.txt'


In [6]:
print("PER for GMM Model with 8 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_8.txt", "./pred/pred_8.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 8 mixtures:
[b'Sentence count: 1671', b'WER:    98.082% (    430047 /     438456)', b'WRR:     4.852% (     21273 /     438456)', b'SER:   100.000% (      1671 /       1671)']


In [7]:
#Test with GMM with 16 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/016"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [8]:
print("Testing with GMM model with 16 mixtures")

pred_file = open("./pred/pred_16.txt","w") 
label_file = open("./pred/label_16.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_16.txt\'")

Testing with GMM model with 16 mixtures
Predictions written to file 'pred_16.txt'


In [9]:
print("PER for GMM Model with 16 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_16.txt", "./pred/pred_16.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 16 mixtures:
[b'Sentence count: 1672', b'WER:    98.443% (    431924 /     438756)', b'WRR:     4.096% (     17972 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [10]:
#Test with GMM with 32 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/032"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [11]:
print("Testing with GMM model with 32 mixtures")

pred_file = open("./pred/pred_32.txt","w") 
label_file = open("./pred/label_32.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_32.txt\'")

Testing with GMM model with 32 mixtures
Predictions written to file 'pred_32.txt'


In [12]:
print("PER for GMM Model with 32 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_32.txt", "./pred/pred_32.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 32 mixtures:
[b'Sentence count: 1672', b'WER:    98.049% (    430195 /     438756)', b'WRR:     4.677% (     20519 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [13]:
#Test with GMM with 64 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/064"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [14]:
print("Testing with GMM model with 64 mixtures")

pred_file = open("./pred/pred_64.txt","w") 
label_file = open("./pred/label_64.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_64.txt\'")

Testing with GMM model with 64 mixtures
Predictions written to file 'pred_64.txt'


In [15]:
print("PER for GMM Model with 64 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_64.txt", "./pred/pred_64.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 64 mixtures:
[b'Sentence count: 1672', b'WER:    98.653% (    432848 /     438756)', b'WRR:     4.026% (     17663 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [16]:
#Test with GMM with 128 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/128"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [17]:
print("Testing with GMM model with 128 mixtures")

pred_file = open("./pred/pred_128.txt","w") 
label_file = open("./pred/label_128.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_128.txt\'")

Testing with GMM model with 128 mixtures
Predictions written to file 'pred_128.txt'


In [18]:
print("PER for GMM Model with 128 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_128.txt", "./pred/pred_128.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 128 mixtures:
[b'Sentence count: 1672', b'WER:    96.239% (    422253 /     438756)', b'WRR:     6.233% (     27348 /     438756)', b'SER:   100.000% (      1672 /       1672)']


In [19]:
#Test with GMM with 256 mixtures without energy coefficient
modelpath = "./models/MFCC_wo_energy_coeff/256"
gmm_files = [os.path.join(modelpath,fname) for fname in os.listdir(modelpath) if fname.endswith('.pkl')]
models = [pickle.load(open(fname,'rb')) for fname in gmm_files]
phonemes   = [fname.split("/")[-1].split(".pkl")[0] for fname in gmm_files]

In [20]:
print("Testing with GMM model with 256 mixtures")

pred_file = open("./pred/pred_256.txt","w") 
label_file = open("./pred/label_256.txt", "w")
id = 1

for full_wav in full_wavs:
    trans, durations = read_transcript(full_wav = full_wav)
    n_delta = 0
    labels = []

    mfcc_features = []
    mfcc_labels = []

    (sample_rate,wav_file) = wav.read(full_wav)
    mfcc_feats = compute_mfcc(wav_file[durations[0][0]:durations[0][1]], n_delta=n_delta)

    for i in range(len(mfcc_feats)):
            labels.append(trans[0])
    
    for index, chunk in enumerate(durations[1:]):
        mfcc_feat = compute_mfcc(wav_file[chunk[0]:chunk[1]], n_delta=n_delta)
        mfcc_feats = np.vstack((mfcc_feats, mfcc_feat))
        for i in range(len(mfcc_feat)):
            labels.append(trans[index])
        #print(mfcc_feats.shape, len(labels))
        
    mfcc_features.extend(mfcc_feats)
    mfcc_labels.extend(labels)
    
    #print(len(mfcc_features), len(mfcc_labels))
    pred = []
    for j in range(len(mfcc_features)): 
       # scores = []
        log_likelihood = np.zeros(len(models))
        
        for i in range(len(models)):
            gmm = models[i]  #checking with each model one by one
            pred_labels = gmm.predict(mfcc_features[j][:-1].reshape(1, -1))
            #print(pred_labels, mfcc_labels[j])
            #scores.append(gmm.score(mfcc_features[j].reshape(1, -1), mfcc_labels[j]))
            #print(scores)
            
            scores = np.array(gmm.score(mfcc_features[j][:-1].reshape(1, -1), mfcc_labels[j]))
            log_likelihood[i] = scores.sum()
            
        #print(np.max(log_likelihood), phonemes[np.argmax(log_likelihood)], mfcc_labels[j], "\n")
        pred.append(phonemes[np.argmax(log_likelihood)])
     
    pred_file.write(str(id) + " " + ' '.join(pred) + "\n")
    label_file.write(str(id) + " " + ' '.join(mfcc_labels) + "\n")
    id += 1
    
print("Predictions written to file \'pred_256.txt\'")

Testing with GMM model with 256 mixtures
Predictions written to file 'pred_256.txt'


In [21]:
print("PER for GMM Model with 256 mixtures:")
wer = subprocess.Popen(["wer", "--head-ids", "./pred/label_256.txt", "./pred/pred_256.txt"], stdout=subprocess.PIPE)
output = wer.communicate()[0]
print(output.splitlines())

PER for GMM Model with 256 mixtures:
[b'Sentence count: 1672', b'WER:    96.191% (    422045 /     438756)', b'WRR:     6.461% (     28350 /     438756)', b'SER:   100.000% (      1672 /       1672)']
